<a href="https://colab.research.google.com/github/rahiakela/transformers-for-natural-language-processing/blob/main/1-model-architecture-of-the-transformer/2_architecture_of_multi_head_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The architecture of multi-head attention

**The multi-head attention sub-layer contains eight heads and is followed by postlayer normalization, which will add residual connections to the output of the sublayer and normalize it.**

<img src='https://github.com/rahiakela/img-repo/blob/master/transformers-for-natural-language-processing/sub-layer-1.png?raw=1' width='800'/>

The input of the multi-attention sub-layer of the first layer of the encoder stack is a vector that contains the embedding and the positional encoding of each word. The next layers of the stack do not start these operations over.

The dimension of the vector of each word $x_n$ of an input sequence is $d_{model} = 512$:

$$
pe(x_n) = [d_1=9.09297407e-01, d_2=9.09297407e-01, .., d_{512}=1.00000000e+00]
$$

The representation of each word $x_n$ has become a vector of $d_{model} = 512$ dimensions.

**Each word is mapped to all the other words to determine how it fits in a sequence.**

In the following sentence, we can see that "it" could be related to "cat" and "rug" in the sequence:

```
Sequence =The cat sat on the rug and it was dry-cleaned.
```

**The model will train to find out if "it" is related to "cat" or "rug."** We could run a huge calculation by training the model using the $d_{model} = 512$ dimensions as they are now.

However, we would only get one point of view at a time by analyzing the sequence
with one $d_{model}$ block. Furthermore, it would take quite some calculation time to find other perspectives.

**A better way is to divide the $d_{model} = 512$ dimensions of each word $x_n$ of $x$ (all of the words of a sequence) into $8 d_k = 64$ dimensions.**

**We then can run the 8 "heads" in parallel to speed up the training and obtain 8 different representation subspaces of how each word relates to another:**

<img src='https://github.com/rahiakela/img-repo/blob/master/transformers-for-natural-language-processing/multi-head-representations.png?raw=1' width='800'/>

**You can see that there are now 8 heads running in parallel.** One head might decide that "it" fits well with "cat" and another that "it" fits well with "rug" and another that "rug" fits well with "dry-cleaned."

The output of each head is a matrix $z_i$ with a shape of $x^*d_k$ The output of a multiattention head is $Z$ defined as:

$$ Z = (z_0, z_1, z_2, z_3, z_4, z_5, z_6, z_7,) $$

**However, $Z$ must be concatenated so that the output of the multi-head sub-layer is not a sequence of dimensions but one lines of $xm*d_{model}$ matrix.**

Before exiting the multi-head attention sub-layer, the elements of $Z$ are concatenated:

$$ MultiHead(output) = Concat(z_0, z_1, z_2, z_3, z_4, z_5, z_6, z_7,) = x, d_{model} $$

**Notice that each head is concatenated into $z$ that has a dimension of $d_{model} = 512$. The output of the multi-headed layer respects the constraint of the original Transformer model.**

Inside each head $h_n$ of the attention mechanism, each word vector has three
representations:

- A query vector $(Q)$ that has a dimension of $d_q = 64$, which is activated and trained when a word vector $x_n$ seeks all of the key-value pairs of the other word vectors, including itself in self-attention
- A key vector $(K)$ that has a dimension of $d_k = 64$, which will be trained to provide an attention value
- A value vector $(V)$ that has a dimension of $d_v = 64$, which will be trained to provide another attention value


Attention is defined as **Scaled Dot-Product Attention** which is represented in the following equation in which we plug $Q$, $K$ and $V$:

$$
Attention(Q,K,V) = softmax \begin{pmatrix} \frac{QK^T}{\sqrt{d_k}} \end{pmatrix} V
$$

**The vectors all have the same dimension making it relatively simple to use a scaled dot product to obtain the attention values for each head and then concatenate the output Z of the 8 heads.**

To obtain $Q$, $K$, and $V$, we must train the model with their respective weight matrices $Q_w, K_w$ and $V_w$, which have $d_k = 64$ columns and $d_{model} = 512$ rows. For example, $Q$ is obtained by a dot-product between $x$ and $Q_w. Q$ will have a dimension of $d_k = 64$.

Hugging Face and Google Brain Trax, among others, provide ready-to-use
frameworks, libraries, and modules. However, let's open the hood of the Transformer model and get our hands dirty in Python to illustrate the architecture we just explored in order to visualize the model in code and show it with intermediate images.

We will use basic Python code with only numpy and a softmax function in 10 steps to run the key aspects of the attention mechanism.

We will start by only using minimal Python functions to understand the Transformer at a low level with the inner workings of an attention head. We will explore the inner workings of the multi-head attention sub-layer using basic code.







## Setup

In [ ]:
# Transformer Installation
!pip -qq install transformers

In [2]:
import numpy as np
from scipy.special import softmax
from transformers import pipeline

## Step 1: Represent the input

The input of the attention mechanism we are building is scaled down to $d_{model} = 4$ instead of $d_{model} = 512$. This brings the dimensions of the vector of an input $x$ down to $d_{model} = 4$, which is easier to visualize.

x contains 3 inputs with 4 dimensions each instead of 512:



In [11]:
print("Step 1: Input : 3 inputs, d_model=4")

x = np.array([
    [1.0, 0.0, 1.0, 0.0],     # Input 1 
    [0.0, 2.0, 0.0, 2.0],     # Input 2 
    [1.0, 1.0, 1.0, 1.0],     # Input 3          
])
print(x.shape)  # (3x4)
print(x)

Step 1: Input : 3 inputs, d_model=4
(3, 4)
[[1. 0. 1. 0.]
 [0. 2. 0. 2.]
 [1. 1. 1. 1.]]


The first step of our model is ready:

<img src='https://github.com/rahiakela/img-repo/blob/master/transformers-for-natural-language-processing/multi-head-input.png?raw=1' width='800'/>

## Step 2: Initializing the weight matrices

We will now add the weight matrices to our model.

Each input has 3 weight matrices:

- $Q_w$ to train the queries
- $K_w$ to train the keys
- $V_w$ to train the values

**These 3 weight matrices will be applied to all the inputs in this model.**

The weight matrices described by **Vaswani** are $d_k = 64$ dimensions.

However, let's scale the matrices down to $d_k = 3$. The dimensions are scaled down to 3*4 weight matrices to be able to visualize the intermediate results more easily and perform dot products with the input $x$.

The three weight matrices are initialized starting with the query weight matrix:

In [12]:
print("Step 2: weights 3 dimensions x d_model=4")
print("query weight matrix")

w_query = np.array([
    [1, 0, 1],
    [1, 0, 0],
    [0, 0, 1],
    [0, 1, 1]                
])
print(w_query.shape)  # (4x3)
print(w_query)

Step 2: weights 3 dimensions x d_model=4
query weight matrix
(4, 3)
[[1 0 1]
 [1 0 0]
 [0 0 1]
 [0 1 1]]


We will now initialize the key weight matrix:

In [13]:
print("key weight matrix")

w_key = np.array([
    [0, 0, 1],
    [1, 1, 0],
    [0, 1, 0],
    [1, 1, 0]                
])
print(w_key.shape)  # (4x3)
print(w_key)

key weight matrix
(4, 3)
[[0 0 1]
 [1 1 0]
 [0 1 0]
 [1 1 0]]


Finally, we initialize the value weight matrix:

In [14]:
print("value weight matrix")

w_value = np.array([
    [0, 2, 0],
    [0, 3, 0],
    [1, 0, 3],
    [1, 1, 0]                
])
print(w_value.shape)  # (4x3)
print(w_value)

value weight matrix
(4, 3)
[[0 2 0]
 [0 3 0]
 [1 0 3]
 [1 1 0]]


The second step of our model is ready:

<img src='https://github.com/rahiakela/img-repo/blob/master/transformers-for-natural-language-processing/multi-head-weight-matrices.png?raw=1' width='800'/>

## Step 3: Matrix multiplication to obtain Q, K, V

We will now multiply the input vectors by the weight matrices to obtain a query,
key, and value vector for each input.

In this model, we will assume that there is one w_query, w_key, and w_value weight matrix for all inputs.

Let's first multiply the input vectors by the w_query weight matrix:



In [15]:
print("Step 3: Matrix multiplication to obtain Q,K,V")

print("Query: x * w_query")
Q = np.matmul(x, w_query)
print(Q.shape)   # (3x4)(4x3) = (3x3)
print(Q)

Step 3: Matrix multiplication to obtain Q,K,V
Query: x * w_query
(3, 3)
[[1. 0. 2.]
 [2. 2. 2.]
 [2. 1. 3.]]


We now multiply the input vectors by the w_key weight matrix:

In [16]:
print("Key: x * w_key")
K = np.matmul(x, w_key)
print(K.shape)   # (3x4)(4x3) = (3x3)
print(K)

Key: x * w_key
(3, 3)
[[0. 1. 1.]
 [4. 4. 0.]
 [2. 3. 1.]]


Finally, we multiply the input vectors by the w_value weight matrix:

In [17]:
print("Value: x * w_value")
V = np.matmul(x, w_value)
print(V.shape)   # (3x4)(4x3) = (3x3)
print(V)

Value: x * w_value
(3, 3)
[[1. 2. 3.]
 [2. 8. 0.]
 [2. 6. 3.]]


The third step of our model is ready:

<img src='https://github.com/rahiakela/img-repo/blob/master/transformers-for-natural-language-processing/k-q-v.png?raw=1' width='800'/>

We have the Q, K, and V values we need to calculate the attention scores.

## Step 4: Scaled attention scores

The attention head now implements the original Transformer equation:

$$
Attention(Q,K,V) = softmax \begin{pmatrix} \frac{QK^T}{\sqrt{d_k}} \end{pmatrix} V
$$

Step 4 focuses on Q and K:

$$
 \begin{pmatrix} \frac{QK^T}{\sqrt{d_k}} \end{pmatrix}
$$

For this model, we will round $\sqrt{𝑑_𝑘} = \sqrt{3} = 1.75$ to 1 and plug the values into the $Q$ and $K$ part of the equation:

In [18]:
print("Step 4: Scaled Attention Scores")

k_d = 1    # square root of k_d=3 rounded down to 1 for this example
attention_scores = (Q @ K.transpose()) / k_d
print(attention_scores)

Step 4: Scaled Attention Scores
[[ 2.  4.  4.]
 [ 4. 16. 12.]
 [ 4. 12. 10.]]


Step 4 is now complete. For example, the score for $x_1$ is [2,4,4] across the $K$ vectors across the head as displayed:

<img src='https://github.com/rahiakela/img-repo/blob/master/transformers-for-natural-language-processing/scaled-attention-scores.png?raw=1' width='800'/>

The attention equation will now apply softmax to the intermediate scores for each vector.

## Step 5: Scaled softmax attention scores for each vector

We now apply a softmax function to each intermediate attention score. Instead of
doing a matrix multiplication, let's zoom down to each individual vector:

In [19]:
print("Step 5: Scaled softmax attention_scores for each vector")

attention_scores[0] = softmax(attention_scores[0])
attention_scores[1] = softmax(attention_scores[1])
attention_scores[2] = softmax(attention_scores[2])

print(attention_scores[0])
print(attention_scores[1])
print(attention_scores[2])

Step 5: Scaled softmax attention_scores for each vector
[0.06337894 0.46831053 0.46831053]
[6.03366485e-06 9.82007865e-01 1.79861014e-02]
[2.95387223e-04 8.80536902e-01 1.19167711e-01]


Step 5 is now complete. For example, the softmax of the score of $x_1$ for all the keys is:

<img src='https://github.com/rahiakela/img-repo/blob/master/transformers-for-natural-language-processing/softmax-score.png?raw=1' width='800'/>

We can now calculate the final attention values with the complete equation.

## Step 6: The final attention representations

We now can finalize the attention equation by plugging V in:

$$
Attention(Q,K,V) = softmax \begin{pmatrix} \frac{QK^T}{\sqrt{d_k}} \end{pmatrix} V
$$

We will first calculate the attention score of input $x_1$ for Steps 6 and 7. We calculate one attention value for one word vector. When we reach Step 8, we will generalize the attention calculation to the other two input vectors.

To obtain Attention $(Q,K,V)$ for $x_1$, we multiply the intermediate attention score by the 3 value vectors one by one to zoom down into the inner workings of the equation:

In [20]:
print("Step 6: attention value obtained by score1/k_d * V")
print(V[0])
print(V[1])
print(V[2])

print("Attention 1")
attention1 = attention_scores[0].reshape(-1, 1)
attention1 = attention_scores[0][0] * V[0]
print(attention1)

print("Attention 2")
attention2 = attention_scores[0][1] * V[1]
print(attention2)

print("Attention 3")
attention3 = attention_scores[0][2] * V[2]
print(attention3)

Step 6: attention value obtained by score1/k_d * V
[1. 2. 3.]
[2. 8. 0.]
[2. 6. 3.]
Attention 1
[0.06337894 0.12675788 0.19013681]
Attention 2
[0.93662106 3.74648425 0.        ]
Attention 3
[0.93662106 2.80986319 1.40493159]


Step 6 is complete. For example, the 3 attention values for $x_1$ for each input have been calculated:

<img src='https://github.com/rahiakela/img-repo/blob/master/transformers-for-natural-language-processing/attention-representations.png?raw=1' width='800'/>

The attention values now need to be summed up.

## Step 7: Summing up the results